# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS-109A Introduction to Data Science 


## Fine Particulate Air Pollution and COVID-19

**Harvard University**<br>
**Spring 2020**<br>
Jack Luby, Hakeem Angulu, and Louie Ayre <br>

---



### The Problem

Fine particulate matter (PM$_{2.5}$) is an air pollutant which has been shown to increase the risk of mortality and hospitalization in exposed populations. 

The fine inhalable particles of PM$_{2.5}$ impact communitites at the local level, incrementally decreasing their life expectancies as ambient concentrations of the pollutant rise. Despite their adverse effects, concentrations of PM$_{2.5}$ are not well monitored throughout much of the United States (especially in regions of low population density). As a result, many communities (and their care providers) are unaware of the life-shortening ambient concentrations of PM$_{2.5}$ they breathe each day. 

These risks have been heightened by the coronavirus pandemic. COVID-19 is primarily a respiratory disease, and PM$_{2.5}$'s adverse effects on respiratory potential have been theorized and shown to increase the likelihood of developing COVID-19. It is apparent that high concentrations of PM$_{2.5}$ are likely to be founded in rural and low-income communities and communities of color. In addition, the US healthcare and social systems have long underserved those communities. The combination of these factors creates an especially bad prognosis for those communities, and necessitates further study and rapid policy and healthcare interventions.

### Our Project

With PM$_{2.5}$ data and COVID-19 data at hand, this project seeks to understand the relationships between PM$_{2.5}$ pollution, demographic and socioeconomic factors, and COVID-19, with a particular focus on rural and low-income communities, and communities of color.

In [2]:
## Set formatting to CS109 standard
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [3]:
# The classics
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt

## Import and Prepare Data

### PM_25 Data

These data were obtained from the __Exposure to air pollution and COVID-19 mortality in the United States__ data repository, found [here](https://github.com/wxwx1993/PM_COVID).

In [148]:
# Import sample airpred data
no_loc_airpred = pd.read_csv("data/airpred.csv")
locations = pd.read_csv("data/airpred_monitor_locations.csv")
no_loc_airpred

,site,year,date,MonitorData,GFEDFireCarbon,USElevation_dsc10000,USElevation_max100,USElevation_max10000,USElevation_mea100,USElevation_mea10000,...,Nearby_Peak2Lag3_MeanTemperature,Nearby_Peak2Lag3_MinTemperature,OMAEROe_UVAerosolIndex_Mean,OMAEROe_VISAerosolIndex_Mean,OMAERUVd_UVAerosolIndex_Mean,OMNO2d_ColumnAmountNO2StratoCloudScreened_Mean,OMO3PR,OMSO2e_ColumnAmountSO2_PBL_Mean,OMTO3e_ColumnAmountO3,OMUVBd_UVindex_Mean
0,1,2012,2012-01-01,-0.128523,0.000000,-0.550570,-0.507102,-0.560471,-0.508208,-0.551128,...,0.786356,0.792680,1.131234,0.161272,-0.095718,0.401790,0.131495,0.326639,-0.902770,1.226424
1,2,2012,2012-01-01,0.038557,0.015970,0.291582,0.308239,0.290428,0.311206,0.291511,...,0.048029,-0.093466,0.367454,1.146508,0.311956,0.351098,0.158020,1.152751,-0.625212,0.626048
2,3,2012,2012-01-01,-0.192785,0.000000,-0.184808,-0.143466,-0.190860,-0.139900,-0.184741,...,0.025757,-0.055311,0.674541,-0.686826,-0.188018,0.289347,-0.165317,-0.180775,-0.251555,0.700011
3,4,2012,2012-01-01,-0.025705,0.000000,0.261766,0.365057,0.259904,0.368308,0.261815,...,0.199577,0.094038,-0.947507,-0.170926,0.052881,0.313887,-0.122894,-0.515723,-0.401922,0.731628
4,5,2012,2012-01-01,0.424127,0.006894,-0.149374,-0.106534,-0.145645,-0.111349,-0.149412,...,-0.007092,-0.117680,-0.690289,0.201874,0.052881,0.324737,-0.050169,0.278625,-0.478801,0.731628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21555,2152,2012,2012-01-10,NaN,0.000000,3.725388,3.731534,3.699302,3.763026,3.724211,...,-1.176077,-1.127496,1.275591,1.426462,1.096113,-0.816608,0.159537,NaN,0.819672,-0.483293
21556,2153,2012,2012-01-10,NaN,0.000000,5.077187,4.845170,5.060265,4.865096,5.075839,...,-0.669501,-0.809156,NaN,NaN,1.450105,-0.816608,-0.323094,NaN,NaN,0.035223
21557,2154,2012,2012-01-10,NaN,0.000000,6.935282,8.115057,6.937766,8.148465,6.933330,...,-1.027416,-1.247245,-0.173228,0.244179,-0.144025,-0.816608,1.026874,0.758817,0.208592,0.383809
21558,2155,2012,2012-01-10,NaN,0.000000,0.372071,0.595170,0.369462,0.605282,0.371799,...,-1.316017,-1.250595,0.007874,0.721465,-0.253582,-0.816608,0.962763,NaN,0.090447,-0.658142


In [149]:
# Merge location data
airpred = no_loc_airpred.merge(locations, how='left')
airpred

,site,year,date,MonitorData,GFEDFireCarbon,USElevation_dsc10000,USElevation_max100,USElevation_max10000,USElevation_mea100,USElevation_mea10000,...,OMAEROe_UVAerosolIndex_Mean,OMAEROe_VISAerosolIndex_Mean,OMAERUVd_UVAerosolIndex_Mean,OMNO2d_ColumnAmountNO2StratoCloudScreened_Mean,OMO3PR,OMSO2e_ColumnAmountSO2_PBL_Mean,OMTO3e_ColumnAmountO3,OMUVBd_UVindex_Mean,Lon,Lat
0,1,2012,2012-01-01,-0.128523,0.000000,-0.550570,-0.507102,-0.560471,-0.508208,-0.551128,...,1.131234,0.161272,-0.095718,0.401790,0.131495,0.326639,-0.902770,1.226424,-87.88026,30.49748
1,2,2012,2012-01-01,0.038557,0.015970,0.291582,0.308239,0.290428,0.311206,0.291511,...,0.367454,1.146508,0.311956,0.351098,0.158020,1.152751,-0.625212,0.626048,-85.80361,33.28493
2,3,2012,2012-01-01,-0.192785,0.000000,-0.184808,-0.143466,-0.190860,-0.139900,-0.184741,...,0.674541,-0.686826,-0.188018,0.289347,-0.165317,-0.180775,-0.251555,0.700011,-87.63810,34.76262
3,4,2012,2012-01-01,-0.025705,0.000000,0.261766,0.365057,0.259904,0.368308,0.261815,...,-0.947507,-0.170926,0.052881,0.313887,-0.122894,-0.515723,-0.401922,0.731628,-85.96986,34.28857
4,5,2012,2012-01-01,0.424127,0.006894,-0.149374,-0.106534,-0.145645,-0.111349,-0.149412,...,-0.690289,0.201874,0.052881,0.324737,-0.050169,0.278625,-0.478801,0.731628,-85.99265,33.99149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21555,2152,2012,2012-01-10,NaN,0.000000,3.725388,3.731534,3.699302,3.763026,3.724211,...,1.275591,1.426462,1.096113,-0.816608,0.159537,NaN,0.819672,-0.483293,-113.03330,43.65000
21556,2153,2012,2012-01-10,NaN,0.000000,5.077187,4.845170,5.060265,4.865096,5.075839,...,NaN,NaN,1.450105,-0.816608,-0.323094,NaN,NaN,0.035223,-119.96670,38.93330
21557,2154,2012,2012-01-10,NaN,0.000000,6.935282,8.115057,6.937766,8.148465,6.933330,...,-0.173228,0.244179,-0.144025,-0.816608,1.026874,0.758817,0.208592,0.383809,-106.74000,40.44500
21558,2155,2012,2012-01-10,NaN,0.000000,0.372071,0.595170,0.369462,0.605282,0.371799,...,0.007874,0.721465,-0.253582,-0.816608,0.962763,NaN,0.090447,-0.658142,-92.83030,48.41320


In [6]:
# County PM data
init_county_pm = pd.read_csv("data/county_pm25.csv")

In [7]:
init_county_pm.head()

,fips,year,pm25
0,36103.0,2000,13.749745
1,36103.0,2001,13.681471
2,36103.0,2002,12.549986
3,36103.0,2003,12.436192
4,36103.0,2004,11.738100


In [8]:
# Drop the counties with no data and no fips
county_pm = init_county_pm.dropna()

In [9]:
# Convert fips to int
county_pm['fips'] = county_pm['fips'].astype(int)

//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [112]:
county_pm.head()

,fips,year,pm25
count,52627.000000,52627.000000,52627.000000
mean,30604.634997,2008.003135,8.400405
std,14954.618736,4.897638,2.954682
min,1001.000000,2000.000000,0.420434
25%,19041.000000,2004.000000,6.168354
50%,29203.000000,2008.000000,8.309108
75%,45083.000000,2012.000000,10.459494
max,56045.000000,2016.000000,22.015262


### Socioeconomic, Demographic, and Behavioral Risk Factor Data

These data were obtained from the __Exposure to air pollution and COVID-19 mortality in the United States__ data repository, found [here](https://github.com/wxwx1993/PM_COVID).

In [11]:
# County socioeconomic and demographic data
init_county_demo = pd.read_csv("data/census_county_interpolated.csv")

In [12]:
# Drop unnecessary axis
init_county_demo.drop(['Unnamed: 0'], axis=1, inplace=True)

In [13]:
init_county_demo.head()

,fips,year,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,36103.0,2000,0.058031,1875.609065,239247.243803,0.047576,66265.088050,0.808135,0.072071,0.246606,1.450411e+06,0.018570,0.002206,0.893031
1,36103.0,2004,0.056872,1883.031226,269377.542545,0.047693,68979.083857,0.811235,0.075708,0.234804,1.457034e+06,0.019281,0.002208,0.892249
2,36103.0,2008,0.047349,1942.548304,506122.762671,0.048704,90873.962264,0.833869,0.105376,0.135350,1.511674e+06,0.025127,0.002184,0.886210
3,36103.0,2011,0.047716,1945.335549,510830.523810,0.048919,90295.504762,0.835889,0.104793,0.139874,1.494680e+06,0.024687,0.002195,0.885766
4,36103.0,2014,0.049384,1956.409276,481809.390476,0.049218,92309.723810,0.824198,0.110059,0.120635,1.508614e+06,0.027491,0.002044,0.880521


In [14]:
# Drop the counties with no fips
county_demo = init_county_demo.dropna()

In [15]:
# Convert fips to int
county_demo['fips'] = county_demo['fips'].astype(int)

//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [86]:
county_demo.head()

,fips,year,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,36103,2000,0.058031,1875.609065,239247.243803,0.047576,66265.088050,0.808135,0.072071,0.246606,1.450411e+06,0.018570,0.002206,0.893031
1,36103,2004,0.056872,1883.031226,269377.542545,0.047693,68979.083857,0.811235,0.075708,0.234804,1.457034e+06,0.019281,0.002208,0.892249
2,36103,2008,0.047349,1942.548304,506122.762671,0.048704,90873.962264,0.833869,0.105376,0.135350,1.511674e+06,0.025127,0.002184,0.886210
3,36103,2011,0.047716,1945.335549,510830.523810,0.048919,90295.504762,0.835889,0.104793,0.139874,1.494680e+06,0.024687,0.002195,0.885766
4,36103,2014,0.049384,1956.409276,481809.390476,0.049218,92309.723810,0.824198,0.110059,0.120635,1.508614e+06,0.027491,0.002044,0.880521


In [17]:
# County behavioral risk factor data
county_brf = pd.read_csv("data/brfss_county_interpolated.csv")

### COVID-19 Data

These data were obtained from the __COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University__, found [here](https://github.com/CSSEGISandData/COVID-19).

In [18]:
def covid_data_prep(date):
    """
    Process COVID-19 county-wise data.
    
    date (string): the date of the data
    """
    init_county_covid = pd.read_csv(f"data/county_covid_{date}.csv")
    
    # Select columns of interest
    select_county_covid = init_county_covid[['FIPS', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
    
    # Add a date column
    select_county_covid['date'] = date
    
    # Drop NA
    select_county_covid.dropna(inplace=True)
    
    # Convert fips to int and format the columns
    select_county_covid['FIPS'] = select_county_covid['FIPS'].astype(int)
    return select_county_covid.rename(str.lower, axis=1)

In [19]:
# Available dates
available_dates = ['03_25', '04_05', '04_15', '04_25', '05_05']

In [20]:
all_county_covid_data = [covid_data_prep(date) for date in available_dates]

//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
//anaconda3/envs/cs109b/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [22]:
long_county_covid = pd.concat(all_county_covid_data)

In [23]:
long_county_covid.head()

,fips,confirmed,deaths,recovered,active,date
0,45001,3,0,0,0,03_25
1,22001,2,0,0,0,03_25
2,51001,2,0,0,0,03_25
3,16001,24,0,0,0,03_25
4,19001,1,0,0,0,03_25


### COVID-19 State Policy Data


In [24]:
# Read CSV
init_policy = pd.read_csv("data/state_policy0410.csv")[:-3]
init_policy.head()

,State,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,Closed non-essential businesses,Religious Gatherings Exempt Without Clear Social Distance Mandate*,Alcohol/Liquor Stores Open,Keep Firearms Sellers Open,...,Extend the amount of time an individual can be on unemployment insurance,Paid sick leave,Population density per square miles,Population 2018,Square Miles,Number Homeless (2019),Percent Unemployed (2018),Percent living under the federal poverty line (2018),Percent at risk for serious illness due to COVID,All-cause deaths 2016
0,Alabama,3/13/2020,3/18/2020,3/20/2020,3/19/2020,4/4/2020,3/28/2020,0,1.0,1.0,...,0.0,0,93.24,4887871,52420.0,"3,261",5.6,16.8,43.1,"52,466"
1,Alaska,3/11/2020,3/16/2020,0,0,3/28/2020,3/28/2020,0,1.0,1.0,...,0.0,0,1.11,737438,665384.0,"1,907",6.8,10.9,32.8,"4,494"
2,Arizona,3/11/2020,3/16/2020,0,0,3/31/2020,0,1,1.0,1.0,...,0.0,1,62.91,7171646,113990.0,"10,007",5.4,14,39.1,"56,645"
3,Arkansas,3/11/2020,3/17/2020,0,3/13/2020,0,0,0,1.0,1.0,...,0.0,0,56.67,3013825,53179.0,"2,717",4.5,17.2,43.5,"31,756"
4,California,3/4/2020,0,0,0,3/19/2020,0,0,1.0,0.0,...,0.0,1,241.65,39557045,163695.0,"151,278",5.5,12.8,33.3,"262,240"


In [72]:
init_policy.columns

Index(['State', 'State of emergency', 'Date closed K-12 schools',
       'Closed day cares', 'Date banned visitors to nursing homes',
       'Stay at home/ shelter in place', 'Closed non-essential businesses',
       'Religious Gatherings Exempt Without Clear Social Distance Mandate*',
       'Alcohol/Liquor Stores Open', 'Keep Firearms Sellers Open',
       'Closed restaurants except take out', 'Closed gyms',
       'Closed movie theaters', 'Froze evictions',
       'Order freezing utility shut offs', 'Froze mortgage payments',
       'Waived one week waiting period for unemployment insurance',
       'Waive work search requirement for unemployment insurance',
       'Expand eligibility of unemployment insurance to anyonewho is quarantined and/or taking care of someone who is quarantined',
       'Expand eligibility of unemployment insurance to those who have lost childcare/school closures',
       'Extend the amount of time an individual can be on unemployment insurance',
       'Pai

In [79]:
def days_since(data=init_policy, col='State of emergency'):
    """
    Extract the number of days since policy changes were enacted.
    
    data(DF): the data
    col(str): the column name
    """
    # Initialize days list
    days = []
    
    # Specify date format
    date_format = "%m/%d/%Y"
    
    # Get the current date
    a = dt.today()
    
    # Take the difference
    for date in data[col]:
        if type(date) != str:
            days.append(0)
        elif len(date) < 8:
            days.append(0)
        else:
            b = dt.strptime(date, date_format)
            delta = a - b
            days.append(delta.days)
    return days

In [267]:
# Build State Policy DF
policy = pd.DataFrame(init_policy.copy()['State'])

cols = init_policy.columns[1:15]
day_cols = list(init_policy.columns[1:7]) + list(init_policy.columns[10:15])
for col in cols:
    if col in day_cols:
        policy[col] = days_since(col=col)
    else:
        policy[col] = [int(i) for i in init_policy[col]]
    
policy.head()

,State,State of emergency,Date closed K-12 schools,Closed day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,Closed non-essential businesses,Religious Gatherings Exempt Without Clear Social Distance Mandate*,Alcohol/Liquor Stores Open,Keep Firearms Sellers Open,Closed restaurants except take out,Closed gyms,Closed movie theaters,Froze evictions,Order freezing utility shut offs
0,Alabama,57,52,50,51,35,42,0,1,1,51,42,42,0,0
1,Alaska,59,54,0,0,42,42,0,1,1,52,52,52,0,0
2,Arizona,59,54,0,0,39,0,1,1,1,50,50,50,46,0
3,Arkansas,59,53,0,57,0,0,0,1,1,50,50,0,0,0
4,California,66,0,0,0,51,0,0,1,0,0,0,0,43,0


#### Note: Some data that are missing may be worth inserting manually, namely CA for K-12


## Combine


In [189]:
# check how many counties are not reported in 2016
len(county_pm.fips.unique()) - len(county_pm.fips.iloc[np.where(county_pm.year == 2016)[0]].unique())

0

In [168]:
county_pm_2016 = pd.DataFrame(county_pm.iloc[np.where(county_pm.year == 2016)[0]][['fips','pm25']]).reset_index(drop=True)
county_pm_2016.head()

,fips,pm25
0,36103,7.927780
1,25013,5.813523
2,25015,4.537500
3,25027,4.716725
4,25003,4.036652


In [190]:
# check how many counties are not reported in 2018
len(county_demo.fips.unique()) - len(county_demo.fips[np.where(county_demo.year == 2018)[0]].unique())

5

5 counties that are otherwise included are missing for 2018

In [175]:
county_demo_2018 = pd.DataFrame(county_demo.iloc[np.where(county_demo.year == 2018)[0]]).reset_index(drop=True).drop(['year'], axis = 1)
county_demo_2018.head()

,fips,poverty,popdensity,medianhousevalue,pct_blk,medhouseholdincome,pct_owner_occ,hispanic,education,population,pct_asian,pct_native,pct_white
0,31039,0.069696,15.759176,115200.0,0.000779,55190.0,0.692431,0.099099,0.189373,8991.0,0.003114,0.001668,0.941497
1,53069,0.078133,15.932669,207000.0,0.012891,54085.0,0.858460,0.063022,0.135608,4189.0,0.015517,0.012175,0.920506
2,35011,0.167570,0.886743,95600.0,0.016019,31028.0,0.663830,0.460680,0.238757,2060.0,0.002913,0.000000,0.745631
3,31109,0.131295,370.232036,167900.0,0.040307,58009.0,0.600256,0.068624,0.137807,310094.0,0.042623,0.005714,0.864054
4,31129,0.121625,7.432776,70800.0,0.001637,42049.0,0.757053,0.027135,0.147681,4275.0,0.013333,0.001637,0.977778


In [199]:
# duplicate spotted
len(all_county_covid_data[-1].fips) - len(all_county_covid_data[-1].fips.unique())

1

In [212]:
# find duplicate
seen = []
duplicate = []

for item in list(np.sort(all_county_covid_data[-1].fips)):
    if item not in seen:
        seen.append(item)
    else:
        duplicate.append(item)
print(duplicate)

[90049]


In [210]:
all_county_covid_data[-1].iloc[list(np.where(all_county_covid_data[-1].fips == duplicate[0])[0])]

,fips,confirmed,deaths,recovered,active,date
2420,90049,121,3,0,118,05_05
2655,90049,0,0,0,0,05_05


In [221]:
all_county_covid_data_5_05 = all_county_covid_data[-1].drop(2655).drop(['date'],axis=1)
all_county_covid_data_5_05.head()

,fips,confirmed,deaths,recovered,active
0,45001,33,0,0,33
1,22001,136,10,0,126
2,51001,429,7,0,422
3,16001,713,19,0,694
4,19001,2,0,0,2


In [222]:
num_pm = len(county_pm_2016.fips.unique())
num_demo = len(county_demo_2018.fips.unique())
num_covid = len(all_county_covid_data_5_05.fips.unique())

print('There are ' + str(num_pm) + ' distinct counties with pm data.')
print('There are ' + str(num_demo) + ' distinct counties with demographic data.')
print('There are ' + str(num_covid) + ' distinct counties with the most recent covid data.')

There are 3097 distinct counties with pm data.
There are 3220 distinct counties with demographic data.
There are 2933 distinct counties with the most recent covid data.


In [225]:
set_pm = set(county_pm_2016.fips.unique())
set_demo = set(county_demo_2018.fips.unique())
set_covid = set(all_county_covid_data_5_05.fips.unique())

county_list = list(np.sort(list(set_pm & set_demo & set_covid)))
num_counties = len(county_list)

print('There are ' + str(num_counties) + ' distinct counties in common across all 3 datasets.')

There are 2841 distinct counties in common across all 3 datasets.


In [292]:
fips_ref = pd.read_csv("data/NCHSURCodes2013.csv").rename(str.lower, axis=1)
state_ref = pd.read_csv("data/statecode.csv").rename({'Code':'state abr.'}, axis='columns')

temp_df_1 = pd.DataFrame(county_list, columns = ['fips']).merge(fips_ref[['fips', 'state abr.']], how = 'left')
temp_df_2 = temp_df_1.merge(state_ref[['State','state abr.']], how = 'left').drop(['state abr.'], axis = 1)
temp_df_3 = temp_df_2.merge(county_pm_2016, how = 'left')
temp_df_4 = temp_df_3.merge(county_demo_2018, how = 'left')
temp_df_5 = temp_df_4.merge(all_county_covid_data_5_05.drop(['active'],axis=1), how = 'left')
main_df = temp_df_5.merge(policy, how = 'left').drop(['State'], axis = 1)

main_df.columns


Index(['fips', 'pm25', 'poverty', 'popdensity', 'medianhousevalue', 'pct_blk',
       'medhouseholdincome', 'pct_owner_occ', 'hispanic', 'education',
       'population', 'pct_asian', 'pct_native', 'pct_white', 'confirmed',
       'deaths', 'recovered', 'State of emergency', 'Date closed K-12 schools',
       'Closed day cares', 'Date banned visitors to nursing homes',
       'Stay at home/ shelter in place', 'Closed non-essential businesses',
       'Religious Gatherings Exempt Without Clear Social Distance Mandate*',
       'Alcohol/Liquor Stores Open', 'Keep Firearms Sellers Open',
       'Closed restaurants except take out', 'Closed gyms',
       'Closed movie theaters', 'Froze evictions',
       'Order freezing utility shut offs'],
      dtype='object')

In [293]:
cols_to_norm = ['pm25', 'poverty', 'popdensity', 'medianhousevalue', 'pct_blk',
       'medhouseholdincome', 'pct_owner_occ', 'hispanic', 'education',
       'population', 'pct_asian', 'pct_native', 'pct_white', 'confirmed',
       'deaths', 'recovered', 'State of emergency', 'Date closed K-12 schools',
       'Closed day cares', 'Date banned visitors to nursing homes',
       'Stay at home/ shelter in place', 'Closed non-essential businesses',
       'Religious Gatherings Exempt Without Clear Social Distance Mandate*',
       'Alcohol/Liquor Stores Open', 'Keep Firearms Sellers Open',
       'Closed restaurants except take out', 'Closed gyms',
       'Closed movie theaters', 'Froze evictions',
       'Order freezing utility shut offs']

temp = main_df[cols_to_norm].values
min_max_scaler = preprocessing.MinMaxScaler()
temp_scaled = min_max_scaler.fit_transform(temp)
main_df[cols_to_norm] = pd.DataFrame(temp_scaled)

In [294]:
main_df.columns = ['fips', 'pm25', 'poverty', 'pop_density', 'med_house_value', 
                     'pct_blk', 'med_household_income', 'pct_owner_occ', 'pct_hispanic', 
                     'education', 'population', 'pct_asian', 'pct_native', 'pct_white', 
                     'confirmed_covid_cases', 'covid_deaths', 'covid_recoveries', 
                     'days_since_state_of_emergency', 'days_since_closing_K-12', 
                     'days_since_closing_day_cares', 'days_since_banning_visitors_to_nursing_homes',
                     'days_since_shelter_in_place', 'days_since_closing_non-essential_businesses',
                     'religious_gatherings_exempt', 'liquor_stores_open', 'firearms_sellers_open',
                     'days_since_closing_restaurants_except_takeout', 'days_since_closing_gyms', 'days_since_closing_movie_theaters', 
                     'froze_evictions', 'order_freezing_utility_shutoffs']
main_df.head()

,fips,pm25,poverty,pop_density,med_house_value,pct_blk,med_household_income,pct_owner_occ,pct_hispanic,education,...,days_since_shelter_in_place,days_since_closing_non-essential_businesses,religious_gatherings_exempt,liquor_stores_open,firearms_sellers_open,days_since_closing_restaurants_except_takeout,days_since_closing_gyms,days_since_closing_movie_theaters,froze_evictions,order_freezing_utility_shutoffs
0,1001,0.671200,0.247670,0.001286,0.117213,0.218957,0.332512,0.743615,0.027941,0.305812,...,0.686275,0.823529,0.0,1.0,1.0,0.927273,0.777778,0.777778,0.0,0.0
1,1003,0.586507,0.156516,0.001814,0.160143,0.108646,0.308184,0.724934,0.045366,0.263580,...,0.686275,0.823529,0.0,1.0,1.0,0.927273,0.777778,0.777778,0.0,0.0
2,1005,0.633035,0.503097,0.000401,0.060861,0.544269,0.120589,0.545903,0.043301,0.552148,...,0.686275,0.823529,0.0,1.0,1.0,0.927273,0.777778,0.777778,0.0,0.0
3,1007,0.687609,0.221327,0.000499,0.064549,0.254833,0.216678,0.746230,0.024510,0.398816,...,0.686275,0.823529,0.0,1.0,1.0,0.927273,0.777778,0.777778,0.0,0.0
4,1009,0.643753,0.229143,0.001238,0.093443,0.017107,0.245581,0.798278,0.092123,0.451338,...,0.686275,0.823529,0.0,1.0,1.0,0.927273,0.777778,0.777778,0.0,0.0


In [295]:
len(main_df.columns)

31

### 